In [28]:
import fitz  # PyMuPDF
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    text = text.replace('\n', ' ')
    return text

def extract_citation_sentences(text):
    citation_patterns = [
        r'\[[0-9,\s]+\]',
        r'\([A-Z][a-z]+ et al\., \d{4}\)',
        r'\([A-Z][a-z]+, \d{4}\)',
        r'\([A-Z][a-z]+ & [A-Z][a-z]+, \d{4}\)',
        r'\([A-Z][a-z]+, [A-Z][a-z]+, & [A-Z][a-z]+, \d{4}\)',
        r'[A-Z][a-z]+ \(\d{4}\)',
        r'[A-Z][a-z]+ and [A-Z][a-z]+ \(\d{4}\)',
        r'[A-Z][a-z]+ et al\. \(\d{4}\)',
    ]

    combined_pattern = '|'.join(citation_patterns)
    sentences = re.split(r'(?<=[.!?])\s+', text)

    citation_sentences = [
        s for s in sentences
        if re.search(combined_pattern, s) and len(s.split()) >= 5
    ]
    return citation_sentences

def highlight_citation_sentences(pdf_path, output_path):
    doc = fitz.open(pdf_path)
    all_text = extract_text_from_pdf(pdf_path)
    citation_sentences = extract_citation_sentences(all_text)

    for page in doc:
        for sentence in citation_sentences:
            areas = page.search_for(sentence)
            for rect in areas:
                highlight = page.add_highlight_annot(rect)
                highlight.update()

    doc.save(output_path)
    doc.close()


# Contoh pemakaian
pdf_path = 'test.pdf'
output_path = 'highlighted_output.pdf'
highlight_citation_sentences(pdf_path, output_path)


In [15]:
import pandas as pd

In [16]:
sentences = pd.DataFrame(citation_sentences)

In [17]:
sentences.head(10)

,0
0,Kalimat rujukan yang tertulis dalam karya tul...
1,Setiap karya ilmiah mempunyai proses analisis ...
2,Klasifikasi merupakan proses penggolongan a...
3,Dengan hasil tersebut NLP memberikan respo...
4,Beberapa penelitian terdahulu NLP dapat beker...
5,Dengan hasil tersebut NLP memberikan respon y...
6,Hasil penelitian sebelumnya hanya berfokus pad...
7,Selain itu terdapat juga penelitian yang ...
8,Hasil yang didapatkan pada penelitian tersebut...
9,Pada penelitian lain dapat melakukan klasifika...


In [1]:
import pandas as pd

data = pd.read_csv('dataset/data_test_asli.csv')

In [3]:
data['clean_text_translated'] = data['clean_text_translated'].fillna(data['clean_text_translated'].mode()[0])
data['label'] = data['label'].fillna(data['label'].mode()[0])

In [4]:
data.isnull().sum()

clean_text_translated    0
label                    0
dtype: int64

In [5]:
data.to_csv('dataset/data_test_asli.csv')